# Helper script for metacoder
Takes in the exported feature table and create inputs for metacoder - works for SILVA

In [3]:
import pandas as pd
import os

try:
    os.mkdir('../data/metacoder')
except FileExistsError:
    print('directory exist')

directory exist


In [4]:
# load input
df_abundance = pd.read_csv('../data/qiime2/filtered/exported-feature-table/feature-table.tsv', sep='\t', skiprows=1)
df_taxonomy = pd.read_csv('../data/qiime2/filtered/exported-feature-table/taxonomy.tsv', sep='\t')

# split taxonomy files
df_tax_split = pd.DataFrame(columns=['Kingdom', 'Phylum', 'Class', 'Order', 'Family', 'Genus', 'Species', 'Other'])
for num, a in enumerate(df_taxonomy['Taxon']):
    #print(num)
    for i in a.split(';'):
        try:
            level = i.split('__')[0]
            taxa = i.split('__')[1]
        except IndexError:
            df_tax_split.loc[num, 'Other'] = a
        if 'k' in level:
            df_tax_split.loc[num, 'Kingdom'] = taxa
        elif 'p' in level:
            df_tax_split.loc[num, 'Phylum'] = taxa
        elif 'c' in level:
            df_tax_split.loc[num, 'Class'] = taxa
        elif 'o' in level:
            df_tax_split.loc[num, 'Order'] = taxa
        elif 'f' in level:
            df_tax_split.loc[num, 'Family'] = taxa
        elif 'g' in level:
            df_tax_split.loc[num, 'Genus'] = taxa
        elif 's' in level:
            df_tax_split.loc[num, 'Species'] = taxa 
df_tax_split = df_tax_split.drop(columns=['Other'])
df_asv = pd.merge(df_taxonomy, df_tax_split, left_index=True, right_index=True)
first = df_asv.pop('Confidence')
df_asv.insert(9, 'Confidence', first.values)

sample = [i for i in df_abundance.columns][1:]
data = [i.split('_')[0] for i in [i.replace('-','_') for i in sample]]

df_sample = pd.DataFrame(data={'SampleID':sample, 'Sites':data})
df_asv = df_asv.rename(columns={'Feature ID':'OTU_ID'})
df_abundance = df_abundance.rename(columns={'#OTU ID':'OTU_ID'})

df_abundance.to_csv('../data/metacoder/asv_table.tsv', sep='\t', index=False)
df_asv.to_csv('../data/metacoder/tax_assignments.tsv', sep='\t', index=False)
df_sample.to_csv('../data/metacoder/sample_metadata.tsv', sep='\t', index=False)